In [1]:
%load_ext autoreload
%autoreload 2

We need first install tensorflow_model_optimization, then needed tensorflow version, then tf_keras,  then needed tensorflow-io version   

tensorflow-addons ?

In [2]:
import tensorflow as tf

2024-07-26 22:51:10.208200: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-26 22:51:10.211047: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-26 22:51:10.219777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 22:51:10.234390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 22:51:10.238481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 22:51:10.249302: I tensorflow/core/platform/cpu_feature_gu

In [4]:
# import tensorflow_io
# from tensorflow_model_optimization.python.core.keras.compat import keras
# from tensorflow_model_optimization.quantization import keras


In [6]:
import tensorflow as tf
# from tensorflow_model_optimization.python.core.keras.compat import keras
import tensorflow_model_optimization as tfmot
import numpy as np
import matplotlib.pyplot as plt

import sys 
sys.path.append("C:/Users/polin/TinyBirdSounds") 

from create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

In [8]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(train_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.
Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [9]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

2024-07-26 22:53:16.343970: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-26 22:53:25.362421: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:
BASE_MODEL_FILE_NAME = "../time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras"
base_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
base_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 47996, 4)          24        
                                                                 
 max_pooling1d_21 (MaxPooli  (None, 5999, 4)           0         
 ng1D)                                                           
                                                                 
 conv1d_22 (Conv1D)          (None, 5995, 4)           84        
                                                                 
 max_pooling1d_22 (MaxPooli  (None, 749, 4)            0         
 ng1D)                                                           
                                                                 
 conv1d_23 (Conv1D)          (None, 745, 8)            168       
                                                                 
 max_pooling1d_23 (MaxPooli  (None, 93, 8)           

In [14]:
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(base_model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

RuntimeError: Layer conv1d_21:<class 'tf_keras.src.layers.convolutional.conv1d.Conv1D'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

In [17]:
q_aware_model.fit(x_train_np[:500], y_train_np[:500],
                  batch_size=32, epochs=1, validation_split=0.1)

/home/polina/newname/env_for_qat/lib/python3.9/site-packages/tf_keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


15/15 [==============================] - 2s 52ms/step - loss: 0.1859 - accuracy: 0.9378 - val_loss: 0.0477 - val_accuracy: 1.0000


In [18]:
_, baseline_model_accuracy = base_model.evaluate(
    x_test_np, y_test_np, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   x_test_np, y_test_np, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9899497628211975
Quant test accuracy: 0.9597989916801453


In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpumko6fhs/assets


INFO:tensorflow:Assets written to: /tmp/tmpumko6fhs/assets
/home/polina/newname/env_for_qat/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1722026900.742888   24464 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1722026900.742951   24464 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-07-26 22:48:20.743359: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpumko6fhs
2024-07-26 22:48:20.747226: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-07-26 22:48:20.747267: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpumko6fhs
2024-07-26 22:48:20.763674: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-26 22:48:20.767221: I tenso

In [20]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(quantized_tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(quantized_tflite_model, x_test_np, y_test_np, model_format="tf_lite")

Validation dataset:


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.96%
Recall: 96.61%
Precision: 94.61%
F1-score: 95.60%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9555363922845688

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9542238282981039

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 95.98%
Recall: 94.76%
Precision: 93.13%
F1-score: 93.94%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9388691658261668

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9352480878717273


In [21]:
print("For Test dataset:")
evaluate_time_of_prediction(quantized_tflite_model, x_test_np, y_test_np, model_format="tf_lite")

For Test dataset:
Accuracy: 95.98%
Recall: 94.76%
Precision: 93.13%
F1-score: 93.94%

Time to make a prediction for a single data point
Mean: 0.001 seconds
Max: 0.005 seconds
Min: 0.001 seconds


In [23]:
# Save model and print its size
QAT_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_speq_qat.tflite"
open(QAT_MODEL_FILE_NAME, "wb").write(quantized_tflite_model)
print("Model file name: ", QAT_MODEL_FILE_NAME)
convert_bytes(get_file_size(QAT_MODEL_FILE_NAME), "KB")

Model file name:  ../spectrogram_models_from_notebooks/cnn/quantized/cnn_mel_speq_qat.tflite
File size: 30.359 Kilobytes
